In [86]:
!pip install pyTelegramBotAPI

!pip install python-telegram-bot[socks]

!pip install markovify

In [19]:
import pandas as pd
import requests 
import markovify
import random
import json
import csv
from pprint import pprint
import telebot  
import conf 

In [ ]:
#настраиваем данные для вывода предложения машины

In [488]:
provs = pd.read_csv('dal_proverbs.csv', sep='\t', encoding='utf-8')

In [489]:
shuffled = provs.sample(frac=1)
train = ' '.join(shuffled.proverb)

In [490]:
m = markovify.Text(train)

In [ ]:
#настраиваем данные с городами

In [491]:
city_data = {"А" : []}

In [492]:
for i in cities["city"]:
    name = i['name']
    city_data[name[0]] = []

In [493]:
for i in cities["city"]:
    name = i['name']
    city_data[name[0]].append(name)

In [494]:
with open("/Users/mashenka/Documents/TG_Bot/cities.json") as file:
    cities = json.load(file)

In [496]:
#Загружаем и редактируем данные предложений Геймана

In [497]:
with open('Nil_Geyman_texty.txt') as f:
    geyman_file = f.read().split('.')


In [498]:
for i in geyman_file:
    if i != '':
        if not i[0].isalpha():
            geyman_file.pop(i.index(i))
    else:
        geyman_file.pop(i.index(i))

In [ ]:
#подготавливаем глобавльные переменные

In [499]:
data_users = {}
data_users_cities = {}
car_or_person = 0

In [500]:
telebot.apihelper.proxy = conf.PROXY
bot = telebot.TeleBot(conf.TOKEN)  # создаем экземпляр бота

In [502]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    global data_users
    global data_users_cities
    global city_data
    data_users_cities[message.chat.id] = city_data
    data_users[message.chat.id] = 0
    bot.send_message(message.chat.id, "Здравствуйте! Хотите поиграть?")
    bot.send_message(message.chat.id, "/yes /no")
    

In [503]:
@bot.message_handler(commands=['help'])
def send_welcome(message):
    bot.send_message(message.chat.id, "Это бот с разными играми? Хотите поиграть? \n Нажмите /start или /play")

In [504]:
@bot.message_handler(commands=['yes', 'play'])
def games_message(message):
    global data_users
    in_game = data_users[message.chat.id]
    if in_game == 0:
        bot.send_message(message.chat.id, "Выберите игру: \n /goroda \n /ugaday_mashina_ili_chelovek")
        data_users[message.chat.id] = 1
    else:
        
        bot.send_message(message.chat.id, "Вы уже играете")

In [505]:
@bot.message_handler(commands=['no'])
def games_message(message):
    global data_users
    in_game = data_users[message.chat.id]
    if in_game == 0:
        bot.send_message(message.chat.id, "Вы больше не играете :(")
    else:
        data_users[message.chat.id] = 0
    bot.send_message(message.chat.id, "Вот вам картинка ")
    contents = requests.get('https://random.dog/woof.json').json()
    url = contents['url']
    bot.send_photo(chat_id=message.chat.id, photo=url)
    bot.send_message(message.chat.id, "Хотите поиграть? \n /yes /no")

In [506]:
@bot.message_handler(commands=['ugaday_mashina_ili_chelovek'])
def game_guess_car_or_person(message):
    global data_users
    global geyman_file
    in_game = data_users[message.chat.id]
    if in_game == 0:
        bot.send_message(message.chat.id, "Вы не играете в игру")
    else:
        bot.send_message(message.chat.id, "Угадай, кто автор предложения: машина или Гейман?")
        choose = random.randint(1, 2)
        if choose == 1: #1 - это значит мы выводим предложение Геймана
            random_sentence = random.randint(0, len(geyman_file))
            bot.send_message(message.chat.id, geyman_file[random_sentence])
            
        else: #выводим сообщение машины
            bot.send_message(message.chat.id, m.make_short_sentence(100))
        bot.send_message(message.chat.id, "/mashina /Geyman")
    
        global mashina_ili_chelovek
        global send_message_computer_or_person
        send_message_computer_or_person = 1
        mashina_ili_chelovek = choose


In [507]:
@bot.message_handler(commands=['mashina'])
def game_guess_car_or_person_answer_car(message):
    global mashina_ili_chelovek
    global data_users
    in_game = data_users[message.chat.id]
    if send_message_computer_or_person == 1:
        choice = mashina_ili_chelovek
        if choice == 1:
            bot.send_message(message.chat.id, "Вы не угадали(")
        else:
            bot.send_message(message.chat.id, "Ура!Вы угадали!")
        data_users[message.chat.id] = 0
        bot.send_message(message.chat.id, "Хотите сыграть еще раз? \n /yes /no")
        
    else:
        bot.send_message(message.chat.id, "Вы не играете в игру")
        

In [508]:
@bot.message_handler(commands=['Geyman'])
def game_guess_car_or_person_answer_person(message):
    global mashina_ili_chelovek
    global data_users
    in_game = data_users[message.chat.id]
    if send_message_computer_or_person == 1:
        choice = mashina_ili_chelovek
        if choice == 2:
            bot.send_message(message.chat.id, "Вы не угадали(")
        else:
            bot.send_message(message.chat.id, "Ура!Вы угадали!")
        data_users[message.chat.id] = 0
        bot.send_message(message.chat.id, "Хотите сыграть еще раз? \n /yes /no")
    else:
        bot.send_message(message.chat.id, "Вы не играете в игру")
    

In [509]:
@bot.message_handler(commands=['goroda'])
def game_cows_and_bulls(message):
    global data_users
    in_game = data_users[message.chat.id]
    if in_game == 0:
        bot.send_message(message.chat.id, "Вы не играете в игру")
    else:
        data_users[message.chat.id] = 2
        bot.send_message(message.chat.id, 'Напишите город с большой буквы')
        bot.send_message(message.chat.id, 'Если захотите закончить, напишите стоп')
        
        

In [510]:
@bot.message_handler(func=lambda m: True)
def send_len(message):
    global data_users
    global data_users_cities
    global city_data
    user_city_data = data_users_cities[message.chat.id]
    in_game = data_users[message.chat.id]
    
    if in_game == 0:
        bot.send_message(message.chat.id, "Здравствуйте! Я еще не умею общаться(( Хотите сыграть в игру?\n/yes /no")
        
        
    elif in_game == 2:
        
        user_city = str(message.text)
        
        if str(message.text) == 'стоп':
            data_users[message.chat.id] = 0
            bot.send_message(message.chat.id, 'Спасибо за игру')
            bot.send_message(message.chat.id, 'Хотите сыграть еще раз? /yes /no')
            data_users_cities[message.chat.id] = city_data
 
            
        else:
            if (user_city[0] in user_city_data) and (user_city in user_city_data[user_city[0]]):
                random_city = 0
                number = user_city_data[user_city[0]].index(user_city)
                
                user_city_data[number] = ''
                letter_for_computer_city = (user_city[len(user_city)-1]).upper()
                
                while user_city_data[letter_for_computer_city][random_city] == '':
                    random_city = random.randint(0, len(user_city_data[letter_for_computer_city])-1)
                
                bot.send_message(message.chat.id, user_city_data[letter_for_computer_city][random_city])
                user_city_data[letter_for_computer_city][random_city] = 1
                data_users_cities[message.chat.id] = user_city_data
                
            else:
                bot.send_message(message.chat.id, 'Вы ввели не город или ввели с маленькой буквы, попробуйте еще раз')
            
            bot.send_message(message.chat.id, 'Если захотите закончить, напишите стоп')
            
            
            
    elif in_game == 1:
        if str(message.text) == 'стоп':
            data_users[message.chat.id] = 0
            bot.send_message(message.chat.id, 'Спасибо за игру')
            bot.send_message(message.chat.id, 'Хотите сыграть еще раз? /yes /no')
            

In [ ]:
bot.polling(none_stop=True)

2020-06-07 23:44:50,667 (util.py:68 WorkerThread53) ERROR - TeleBot: "ApiException occurred, args=('A request to the Telegram API was unsuccessful. The server returned HTTP 400 Bad Request. Response body:\n[b\'{"ok":false,"error_code":400,"description":"Bad Request: message text is empty"}\']',)
Traceback (most recent call last):
  File "/Users/mashenka/opt/anaconda3/lib/python3.7/site-packages/telebot/util.py", line 62, in run
    task(*args, **kwargs)
  File "<ipython-input-506-6c41deae9244>", line 13, in game_guess_car_or_person
    bot.send_message(message.chat.id, geyman_file[random_sentence])
  File "/Users/mashenka/opt/anaconda3/lib/python3.7/site-packages/telebot/__init__.py", line 625, in send_message
    reply_markup, parse_mode, disable_notification, timeout))
  File "/Users/mashenka/opt/anaconda3/lib/python3.7/site-packages/telebot/apihelper.py", line 159, in send_message
    return _make_request(token, method_url, params=payload, method='post')
  File "/Users/mashenka/opt/